In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [14]:

inputs = tokenizer("<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>A dictionary that maps attention modules to", return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss = outputs.loss
logits = outputs.logits

values , indices = torch.topk(logits[0][-1],k=5)
[tokenizer.decode(x) for x in indices]

[' the', ' a', ' their', ' specific', ' different']

In [16]:

inputs = tokenizer("A dictionary that maps attention modules to", return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss = outputs.loss
logits = outputs.logits

values , indices = torch.topk(logits[0][-1],k=5)
[tokenizer.decode(x) for x in indices]

[' the', ' their', ' a', ' specific', ' different']

In [17]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2', return_dict=True)

# when generating, we will use the logits of right-most token to predict the next token
# so the padding should be on the left
tokenizer.padding_side = "left" 
tokenizer.pad_token = tokenizer.eos_token # to avoid an error


In [20]:

sentences = ["Hello, my dog is a little",
             "A dictionary",
            "Hello, my dog is", # use different length sentences to test batching
            ]
inputs = tokenizer(sentences, return_tensors="pt", padding=True)


output_sequences = model.generate(
    input_ids=inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    do_sample=False, # disable sampling to test if batching affects output
)

for i in range(len(sentences)):
    print(tokenizer.decode(output_sequences[i]))
    # you can use skip_special_tokens=True in decode() to remove padding token
    # but note that it will also remove other special_tokens

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, my dog is a little bit of a mess. I'm not sure if he's going
<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>A dictionary of the most common words in English.

The dictionary is
<|endoftext|><|endoftext|>Hello, my dog is a little bit of a mess. I'm not sure if he


In [22]:
print(inputs['attention_mask'])
print(inputs['input_ids'])

tensor([[1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 1, 1],
        [0, 0, 1, 1, 1, 1, 1]])
tensor([[15496,    11,   616,  3290,   318,   257,  1310],
        [50256, 50256, 50256, 50256, 50256,    32, 22155],
        [50256, 50256, 15496,    11,   616,  3290,   318]])


In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium', return_dict=True)

# when generating, we will use the logits of right-most token to predict the next token
# so the padding should be on the left
tokenizer.padding_side = "left" 
tokenizer.pad_token = tokenizer.eos_token # to avoid an error

In [8]:
import torch
inp = "<|endoftext|>In summary, I believe that the Bible"
mask = torch.tensor([0.] + [1.]*8)
print(mask)
enc = tokenizer([inp],return_tensors='pt')
print(enc)
output = model(input_ids=enc['input_ids'],encoder_attention_mask = mask)
print(output.logits.size())
last = output.logits[0][-1]
print(last.size())
v,i =  last.topk(5)
[tokenizer.decode(x) for x in i]

tensor([0., 1., 1., 1., 1., 1., 1., 1., 1.])
{'input_ids': tensor([[50256,   818, 10638,    11,   314,  1975,   326,   262,  9111]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}
torch.Size([1, 9, 50257])
torch.Size([50257])


[' is', ' should', "'s", ' teaches', ' has']

[' is', ' should', "'s", ' teaches', ' has']